In [ ]:
import pandas as pd

# Load the data
cleaned_data = pd.read_csv(r'Study2_data.csv')

# Convert StartDate to datetime and extract Year-Month
cleaned_data['StartDate'] = pd.to_datetime(cleaned_data['StartDate'])
cleaned_data['YearMonth'] = cleaned_data['StartDate'].dt.to_period('M')

# Combine LSAS_avo and LSAS_anx into LSAS_1, LSAS_2, ..., LSAS_24
for i in range(1, 25):
    avo_col = f'LSAS_avo{i}'
    anx_col = f'LSAS_anx{i}'
    combined_col = f'LSAS_{i}'
    cleaned_data[combined_col] = cleaned_data[avo_col] + cleaned_data[anx_col]

# Drop LSAS_avo_* and LSAS_anx_* columns
cols_to_drop = [f'LSAS_avo{i}' for i in range(1, 25)] + [f'LSAS_anx{i}' for i in range(1, 25)]
print("Columns before dropping:", cleaned_data.columns.tolist())
cleaned_data = cleaned_data.drop(columns=cols_to_drop, errors='raise')  # Use 'raise' to ensure it fails if columns aren't found
print("Columns after dropping:", cleaned_data.columns.tolist())

# Ensure only combined LSAS columns are used
lsas_columns = [f'LSAS_{i}' for i in range(1, 25)]
print("LSAS columns:", lsas_columns)

# Rename BDI1 to BDI_1, BDI2 to BDI_2, ..., BDI21 to BDI_21
for i in range(1, 22):
    old_col = f'BDI{i}'
    new_col = f'BDI_{i}'
    cleaned_data.rename(columns={old_col: new_col}, inplace=True)

# Define other question columns
bdi_columns = [col for col in cleaned_data.columns if 'BDI' in col]
cfs_columns = [col for col in cleaned_data.columns if 'CFS' in col]

# Reverse score CFS questions 2, 3, 5, 10 in place
reverse_columns = ['CFS_2_R', 'CFS_3_R', 'CFS_5_R', 'CFS_10_R']

# Apply the reverse scoring transformation in place
cleaned_data[reverse_columns] = 7 - cleaned_data[reverse_columns]

# Reverse the DIRECTION of all CFS columns
cleaned_data[cfs_columns] *= -1

# Combine all question columns into one list
question_columns = lsas_columns + bdi_columns + cfs_columns
print("Question columns for melt:", question_columns)

# Reshape the data so that each row is a unique observation
# Each row should have a Participant_Number, StartDate, EndDate, Question, and Score
long_data = pd.melt(cleaned_data, 
                    id_vars=['Participant_Number', 'YearMonth'], 
                    value_vars=question_columns, 
                    var_name='Question', value_name='Score')

# Aggregate observations within the same time point by the mean
# Create a dictionary of unique participants for each time point
participant_dict = long_data.groupby('YearMonth')['Participant_Number'].apply(set).to_dict()
long_data= (
    long_data.groupby(['Participant_Number', 'YearMonth', 'Question'])['Score']
    .mean()
    .reset_index()
)

# Standardize
long_data['Score'] = (long_data['Score'] - long_data['Score'].mean()) / long_data['Score'].std()

long_data_checkpoint = long_data
df = long_data_checkpoint

# Select three months based on participant counts and temporal coverage
selected_months = ['2020-01', '2020-05', '2020-09']

# Filter the dataset for the selected months
df['YearMonth'] = df['YearMonth'].astype(str)  # Convert YearMonth to string for filtering
filtered_df = df[df['YearMonth'].isin(selected_months)]

# Aggregate data by participant, month, and test type
aggregated_df = (
    filtered_df.groupby(['Participant_Number', 'YearMonth', 'Question'])['Score']
    .first()
    .reset_index()
)

# Step 6: Reshape data into wide format
wide_df = aggregated_df.pivot_table(
    index='Participant_Number',
    columns=['YearMonth', 'Question'],
    values='Score'
).reset_index()

# Flatten multi-index columns
wide_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in wide_df.columns]

# Make sure Participant_Number_ is renamed to Participant_Number
wide_df.rename(columns={'Participant_Number_': 'Participant_Number'}, inplace=True)

# Display the resulting DataFrame
# Save the reshaped dataset if needed
wide_df.to_csv('OLS_ready.csv', index=False)


Columns before dropping: ['Qualtricsname', 'Participant_Number', 'Corona', 'ResponseID', 'Worker_ID', 'StartDate', 'EndDate', 'Durationinseconds', 'Age', 'Gender', 'Education_version', 'Education_years_A', 'Relationship_status', 'Sexual_attraction', 'Children_YN', 'Children_num', 'Nationality', 'Ethnicity', 'LSAS_anx1', 'LSAS_anx2', 'LSAS_anx3', 'LSAS_anx4', 'LSAS_anx5', 'LSAS_anx6', 'LSAS_anx7', 'LSAS_anx8', 'LSAS_anx9', 'LSAS_anx10', 'LSAS_anx11', 'LSAS_anx12', 'LSAS_anx13', 'LSAS_anx14', 'LSAS_anx15', 'LSAS_anx16', 'LSAS_anx17', 'LSAS_anx18', 'LSAS_anx19', 'LSAS_anx20', 'LSAS_anx21', 'LSAS_anx22', 'LSAS_anx23', 'LSAS_anx24', 'LSAS_avo1', 'LSAS_avo2', 'LSAS_avo3', 'LSAS_avo4', 'LSAS_avo5', 'LSAS_avo6', 'LSAS_avo7', 'LSAS_avo8', 'LSAS_avo9', 'LSAS_avo10', 'LSAS_avo11', 'LSAS_avo12', 'LSAS_avo13', 'LSAS_avo14', 'LSAS_avo15', 'LSAS_avo16', 'LSAS_avo17', 'LSAS_avo18', 'LSAS_avo19', 'LSAS_avo20', 'LSAS_avo21', 'LSAS_avo22', 'LSAS_avo23', 'LSAS_avo24', 'BDI_1', 'BDI_2', 'BDI_3', 'BDI_4', '